# Instagram Follow Bot
author: anthony2owaru@gmail.com

- An established IG account with 2 factor can follow up to 700 accounts without being blocked using this script and the current rate limit settings.
- Using a fresh IG account may require more restrictive rate limiting and may be unable to follow this total per day.
- Some elements may be differently arranged depending on the account you are logged into. This may be a normal function of different UX being served to different accounts. We search elements by tag and text attributes in places to overcome this. 
- We also utilize mouse clicks for repetetive actions to behave more like a human. 
- selenium_stealth package is also used here, see [selenium-stealth](https://github.com/diprajpatra/selenium-stealth) for more info

In [ ]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium_stealth import stealth # https://github.com/diprajpatra/selenium-stealth

import time 
from datetime import datetime 
datetime.now().strftime('%d/%m/%Y %H:%M:%S')
import random

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
#options.add_argument("--headless") 

options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(options=options,
                          executable_path='/Users/am/chromedriver')

#see docs if we need to edit this (https://github.com/diprajpatra/selenium-stealth)
stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

In [ ]:
from random import randint
from time import sleep
import os
        

def mouse_click(element):
    actions = ActionChains(driver)
    actions.move_to_element(element)
    actions.click(element)
    actions.perform()

"""Search Elements by Tag name and text"""
def first_element_by_tag_and_text(tag_name, text_search, button_index=0):
    try:
        elements = driver.find_elements(By.TAG_NAME,tag_name) #creates list of elements
    except NoSuchElementException:
        print("No Such Elements with tag {} Try another search").format(tag_name)
        return None
#    print('found {} elements with tag: {}'.format(len(elements), #debug
#                                                  tag_name))
    elements_with_text_match = [element for element in elements
                                if element.get_attribute('innerText')==str(text_search)
                                or element.get_attribute('outerText')==str(text_search)
                                or element.get_attribute('textContent')==str(text_search) 
                                or element.get_attribute('placeholder')==str(text_search)]#only the elements with innerTextSearch matches
    #print('found {} elements with text: {}, returning index {}'.format(len(elements_with_text_match), #debug
    #                                                                   text_search,
    #                                                                   button_index))
    matched_element = elements_with_text_match[button_index]
    return matched_element
    
    
"""Follows and returns follow status data"""
# the 1st/2nd button logic intends to resolve differences between biz and personal IG accounts (biz has message button first with follow button second when not following whereas personal IG accounts have message as first button when Already followed)
def follow_user(username,action=None): #action = follow or unfollow #for later implementation of follow_action
    data={'username':[],
          'follow_status':[],
          'follow_status_timestamp':[],
          'follow_action':[]}
    buttons = driver.find_elements(By.TAG_NAME,'button') #creates list of buttons, checks the first two buttons of a profile page
    buttons_with_text = [button for button in buttons if button.get_attribute('innerText')!='']#only the buttons with innerText values
    first_button = buttons_with_text[0] #just the first two buttons are significant (down the line, better to just confirm/click Follow button IN <header> without indexing) #just the first two buttons (down the line, better to just confirm/click Follow button IN <header> without indexing)
    second_button = buttons_with_text[1]
    first_button_text = first_button.get_attribute('innerText')
    second_button_text = second_button.get_attribute('innerText')
    print(first_button_text,second_button_text)
    data['username'].append('username')
    data['follow_status'].append(str(first_button_text))
    datetimestamp = datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    data['follow_status_timestamp'].append(datetimestamp)
#   data['follow_action'].append(lower(action)) # when we implement follow_action
        
    if first_button_text == 'Follow': 
        print('[{}] Clicking Follow For User: {}'.format(datetimestamp,
                                                         username))
        first_button.click()
        return data
    if second_button_text == 'Follow': 
        print('[{}] Clicking Follow For User: {}'.format(datetimestamp,
                                                         username))
        second_button.click()
        return data
    if first_button_text == 'Follow Back': #if the 1st button is 'Follow' we follow the user
        print('[{}] User Followed You - Following Back: {}'.format(datetimestamp,
                                                         username))
        first_button.click()
        return data

    elif first_button_text == 'Message' and second_button_text != 'Follow': #if the 1st button is 'Message' we're already following that user | we can build in messaging at this stage too with a function that clicks the message button, and then starts from the typing box of the message
        print('[{}]\tAlready Following User: {}, NOT clicking follow button'.format(datetimestamp,
                                                                                    username))
        return data
    elif first_button_text == 'Requested': #if the 1st button is 'Requested', it was a private account we already tried to follow
        print('[{}]\tAlready Requested User: {} has already been requested! …skipping…'.format(datetimestamp,
                                                                                               username))
        return data
    else: 
        print('[{}]\tUser:{} has no buttons with "", "", or "Follow. Please Check this, URL was: {}"'.format(datetimestamp,
                                                                                                             username,
                                                                                                             driver.current_url))
        return data
    
"""Navigating Profiles ''Normally'' by Using The Search Bar"""

def navigate_profile_search(username):
    time.sleep(3.5)
    search_input_element = first_element_by_tag_and_text('input', 'Search')
    mouse_click(search_input_element)
    driver.switch_to.active_element.clear()
    driver.switch_to.active_element.send_keys(username)
    
    try:
        search_result = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//a[@href="'+'/'+str(username)+'/'+'"]')))
    except TimeoutException:
        print('Result href did not match username: {}, please check your dataset'.format(username))
        return 
    search_result_text = search_result.get_attribute('innerText')
    print(search_result_text)
    if search_result_text == 'No results found.' or search_result is None:
        print('No results for username: {}, please check your dataset'.format(username))
    else:
        time.sleep(3.3)
        mouse_click(search_result)
        

def instagram_login(username,pw):
    driver.get("https://instagram.com")
    time.sleep(2.88)    
    username_field = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
    password_field = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))
    username_field.send_keys(str(username))
    password_field.send_keys(str(pw))
    submit_login = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()




# Logging In

(Please review best practices for storing sensitive keys in Python scripts)

In [ ]:
""""Enter your login information here. Can hold multiple logins. Please review best practices for storing sensitive keys in Python scripts"""

accounts={'YOUR_USERNAME':'YOUR_PASSWORD'}
print('The following accounts are available for login: {}'.format(list(accounts.keys())))

In [ ]:
login = 'YOUR_USERNAME'
instagram_login(login,accounts[login])

# Click "Not Now" for IG startup preferences

In [ ]:
not_now = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Not Now')]"))).click()
not_now2 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Not Now')]"))).click()

# Setting up follow lists
- Down the line this would be better served to function with persistent storage. 
- A great way to use this is to simply take a list of a competitor's following and allow the bot to ingest that list

In [ ]:
# List of currently followed users
# List of usernames you'd like to follow
# usernames_to_follow only contains the latter and excludes the former

followed_users = ['sql_learner_']
usernames_list = ['sql_learner_', 'sql_interview_question', 'sql_database_administrator']
usernames_to_follow = [username for username in usernames_list if username not in followed_users]

#this will take data returned from the follow_user() function below
data = {'index':[],
        'username':[],
        'follow_status':[],
        'follow_status_timestamp':[]

# Spinning up the bot
- Below, we loop through usernames with rate limits (see rate limit notes)

In [ ]:
#### Notes on Rate Limiting ####
# use an established instagram account for best results
# if using a fresh instagram account
# for an established account, the total limit is 700 in 24 hours, 
# The total time.sleep in this program for 700 iterations would be (((700/49)*72) + ((700/2)*45) + (700*10.1))  /60/60 = 6.62Hrs 
# But with other latencies, it will take a full day. 
# This has allowed me to follow up to the daily limit without being blocked. The bot only gets stopped once reaching its daily limit (700 for established accounts). 
# This loop has two rate limits and you can even add a third to make the bot behave a little more naturally


for index, username in enumerate(usernames_to_follow):
    print('starting index # {}/{} for username {}'.format(index,len(usernames_to_follow), username))
    datetimestamp = datetime.now().strftime('%d/%m/%Y %H:%M:%S')

    #This sleeps the bot 45 seconds after every other iteration & 72 seconds every 49 iterations
    
    if index !=0 and index % 49==0: #after every 49 loops
                rate_sleep = 72 # sleep for 72 seconds 
    elif index !=0 and index % 2==0: # for every other loop
        rate_sleep = 45 # sleep for 45 seconds
    #else:
        #rate_sleep = 25 # sleep for 25 seconds in every other loop (this is to set a third time parameter here or leave it if you want)
    print(index,username)
    navigate_profile_search(username)
    time.sleep(3.3)
    follow_data = follow_user(username) #this both runs the function for following the user AND it returns data from that follow, can be expanded to grab other data from each account during the follow sequence
    data['follow_status'].append(follow_data['follow_status'])
    data['follow_status_timestamp'].append(follow_data['follow_status_timestamp'])
    data['username'].append(username)
    followed_users.append(username)
    time.sleep(rate_sleep)
    print('[{}]\tsleeping for {} seconds! …'.format(datetimestamp,
                                                               rate_sleep))
